In [27]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os
import seaborn as sns

In [32]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

ma_pts = ee.FeatureCollection(
    [
        ee.Feature(ee.Geometry.Point([-70.842918395996, 42.730953216553]), {'site_id': 'LP HM'}),
        ee.Feature(ee.Geometry.Point([-70.842468261719, 42.731742858887]), {'site_id': 'LP LM'}) 
    ]
)

fc_all = ma_pts

##Adding every plot coordinate
s_patens = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Massachusetts/LTE-MP-LPP-biomass.csv')
s_alterniflora = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Massachusetts/LTE-MP-LPA-biomass.csv')

s_patens['Latitude'] = 42.730953216553
s_patens['Longitude'] = -70.842918395996

s_alterniflora['Latitude'] = 42.730954
s_alterniflora['Longitude'] = -70.842915

s_patens.rename(columns={'LIVE biomass':'liveMass'}, inplace=True)
s_alterniflora.rename(columns={'MEAN BIOMASS':'liveMass'}, inplace=True)

df = pd.concat([s_patens, s_alterniflora])

df['Date'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# df = df[df['TRT'] == 'C'] ##Remove fertilized plots? Since they won't represent the pixel well. But they might?

dfx =  df.groupby(['SITE', 'Date'], as_index = False).aggregate(
    {
        'Longitude':[np.mean], 'Latitude':[np.mean], 'liveMass':[np.mean, np.size], 
        'SP biomass':[np.mean],'SA biomass':[np.mean], 'MONTH':'first', 'YEAR':'first'
    }
) 

dfx.columns = [
    'Site','Date','Longitude','Latitude','liveMass', 'Sample size','spMass', 'saMass', 'Month', 'Year'
]

display(dfx) ##Grouped by site and date; average biomass 

##NOTE: Structure of data extraction will have to be diff from VA/GA - lag variables/month sampled will change FOR EACH ROW
##in the current dataset (since samlping occurs in different months for each row). Not exactly sure how to do this yet.

,Site,Date,Longitude,Latitude,liveMass,Sample size,spMass,saMass,Month,Year
0,LTE-MP-LPA,1999-06-30,-70.842915,42.730954,578.075000,4,NaN,NaN,6,1999
1,LTE-MP-LPA,1999-07-24,-70.842915,42.730954,486.750000,4,NaN,NaN,7,1999
2,LTE-MP-LPA,1999-08-27,-70.842915,42.730954,389.725000,4,NaN,NaN,8,1999
3,LTE-MP-LPA,1999-09-22,-70.842915,42.730954,441.625000,4,NaN,NaN,9,1999
4,LTE-MP-LPA,1999-10-26,-70.842915,42.730954,211.300000,4,NaN,NaN,10,1999
...,...,...,...,...,...,...,...,...,...,...
210,LTE-MP-LPP,2020-08-02,-70.842918,42.730953,1009.666667,6,644.833333,0.0,8,2020
211,LTE-MP-LPP,2021-05-11,-70.842918,42.730953,57.000000,6,48.333333,0.0,5,2021
212,LTE-MP-LPP,2021-06-22,-70.842918,42.730953,462.000000,6,291.833333,0.0,6,2021
213,LTE-MP-LPP,2021-07-27,-70.842918,42.730953,687.333333,6,543.333333,0.0,7,2021


In [33]:
# sns.lineplot(data = dfx, x = dfx.Date, y = dfx.liveMass)

In [3]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  ## Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

In [4]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

##Notes: This is setup for Landsat 8 - apply to l5?

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

In [5]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30 
    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

##FOR 1m DEM:
def dem1Extraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 1 
    )
    return feature

In [6]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
Map.addLayer(dem1, dem_params, '1m DEM')

In [8]:
##Calculating mean pixel values for time periods within each year

def monthly_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

## Compute monthly averages
# monthly_sowing_Avg = monthly_Avg (ndvi_sowSeason, years, sowingMonths)

##Months and years are lists

years_ls5 = range(2000, 2012)
years_ls7 = range(2012, 2013)
years_ls8 = range(2013, 2021)
years_dm = range(2000, 2021)

months = range(5,11)
months_daymet = range(1,12)

ls5_collect = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(fc_all).map(maskL5sr).map(addFLATSL5).map(maskFLATS)
ls7_collect = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(fc_all).map(maskL5sr).map(addFLATS).map(maskFLATS)
ls8_collect = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(fc_all).map(maskL5sr).map(addFLATS).map(maskFLATS)

monthly_ls5 = monthly_Avg(ls5_collect, years = years_ls5)
monthly_ls7 = monthly_Avg(ls7_collect, years = years_ls7)
monthly_ls8 = monthly_Avg(ls8_collect, years = years_ls8)

monthly_ls5.size().getInfo()

##From stackexchange: calculating monthly averages across many years:
#https://gis.stackexchange.com/questions/290892/google-earth-enginesst-by-month-per-year
#https://gis.stackexchange.com/questions/426662/image-collection-monthly-averages-using-geemap-package

12

In [9]:
list_5 = monthly_ls5.toList(monthly_ls5.size())
list_7 = monthly_ls7.toList(monthly_ls7.size())
list_8 = monthly_ls8.toList(monthly_ls8.size())

ls5_2000 = ee.Image(list_5.get(0))
ls5_2001 = ee.Image(list_5.get(1))
ls5_2002 = ee.Image(list_5.get(2))
ls5_2004 = ee.Image(list_5.get(4)) ##SKIP 2003
ls5_2005 = ee.Image(list_5.get(5))
ls5_2006 = ee.Image(list_5.get(6))
ls5_2007 = ee.Image(list_5.get(7))
ls5_2008 = ee.Image(list_5.get(8))
ls5_2009 = ee.Image(list_5.get(9))
ls5_2010 = ee.Image(list_5.get(10))
ls5_2011 = ee.Image(list_5.get(11))

ls7_2012 = ee.Image(list_7.get(0))

ls8_2013 = ee.Image(list_8.get(0))
ls8_2014 = ee.Image(list_8.get(1))
ls8_2015 = ee.Image(list_8.get(2))
ls8_2016 = ee.Image(list_8.get(3))
ls8_2017 = ee.Image(list_8.get(4))
ls8_2018 = ee.Image(list_8.get(5))
ls8_2019 = ee.Image(list_8.get(6))
ls8_2020 = ee.Image(list_8.get(7))

# ls5_2004.getInfo()

In [ ]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8')

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 5', (df['B4']-df['B3'])/(df['B4']+df['B3']), \
                      (df['B5']-df['B4'])/(df['B5']+df['B4'])) ##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B1'], df['B2'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B2'], df['B3'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B3'], df['B4'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B4'], df['B5'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B5'], df['B6'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B7'], df['B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'va_augoct.csv')
# df.to_csv(out_csv, index = False)